In [1]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
import csv
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
DATA_HOME = 'data'

REVIEWDATA_HOME = os.path.join(DATA_HOME, 'amazon_reviews_us_Camera_v1_00.tsv')

#test = os.path.join(DATA_HOME, 'cs224u-colors-bakeoff-data.csv')
# GLOVE_HOME = os.path.join(DATA_HOME, 'glove.6B')

In [3]:
START_SYMBOL = "<s>"
END_SYMBOL = "</s>"
UNK_SYMBOL = "$UNK"

In [4]:
df = pd.read_csv(REVIEWDATA_HOME, sep='\t',error_bad_lines=False, warn_bad_lines=False)

In [5]:
df.columns.tolist()

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [6]:
# Randomly select a small portion of the entire dataset for model implementation to save computation time
small_df = df.sample(frac=0.01, random_state=1).reset_index(drop=True)
print(small_df.shape)
small_df.head(3)

(18008, 15)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12491786,R1C0EVYBV2Z18Z,B000U92DLA,845576079,Kastar Camera & Camcorder Battery Home Travel ...,Camera,5,6,6,N,Y,A Must For On The Go,"Purchased the charger from Nextop2100, arrived...",2008-11-22
1,US,37280009,R3Q837V78NS6B9,B0008GCYNW,615481480,Olympus Camedia C755 4MP Digital Camera with 1...,Camera,3,15,17,N,Y,The Good and the Not-so Good,I was attracted to this camera for three reaso...,2005-09-08
2,US,16532896,R22CZCJH5YNX1O,B003CZ7L84,884672402,Intova CP-9 Compact Digital Camera with 130 fe...,Camera,4,1,1,N,Y,Great for what we paid!,We took it with us to Cancun. My husband took ...,2011-11-02


In [7]:
# Select features that may by useful for model training
features = ['product_id', 'star_rating', 'helpful_votes', 'total_votes', 'verified_purchase', 'review_headline', 'review_body']
small_df_selectedFeatures = small_df[features]

# Convert 5-star ratings into 2 (positive and negative) or 3 (positive, neutral and negative) classes
def starRating_to_PosNegClass(dataset, num_class):
    if num_class == 2:    
        # Negative reviews: 1-3 Stars = 0 ; Positive reviews: 4-5 Stars = 1
        dataset['pos_neg'] = [1 if x > 3 else 0 for x in dataset.star_rating]
    if num_class == 3: 
        # Negative reviews: 1-2 Stars = -1 ; Positive reviews: 4-5 Stars = 1 ; Neutral Reviews: 3 Stars = 0
        dataset['pos_neg_1'] = [1 if x > 3 else 0 for x in dataset.star_rating] 
        dataset['pos_neg_2'] = [-1 if x < 3 else 0 for x in dataset.star_rating] 
        dataset['pos_neg'] = (dataset['pos_neg_1'] + dataset['pos_neg_2'])
        dataset.drop(['pos_neg_1', 'pos_neg_2'], axis=1)
    return dataset


small_df_selectedFeatures = starRating_to_PosNegClass(small_df_selectedFeatures, num_class=2)
print('# of positive reviews and negative reviews: ')
small_df_selectedFeatures['pos_neg'].value_counts()

# of positive reviews and negative reviews: 


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


1    13925
0     4083
Name: pos_neg, dtype: int64

In [8]:
small_df_selectedFeatures.head(5)

,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,pos_neg
0,B000U92DLA,5,6,6,Y,A Must For On The Go,"Purchased the charger from Nextop2100, arrived...",1
1,B0008GCYNW,3,15,17,Y,The Good and the Not-so Good,I was attracted to this camera for three reaso...,0
2,B003CZ7L84,4,1,1,Y,Great for what we paid!,We took it with us to Cancun. My husband took ...,1
3,B000EB7OTU,5,0,1,N,Great Bag,This bag is perfect for our nikon d80. It has...,1
4,B00MIB0JG4,1,0,0,Y,CRAP Don't waist the time and money,"Crapy cheap plastic, leg broke the second I to...",0


In [92]:
# CountVectorizer -> 應該是 bag of word? 不知道我有沒有理解錯誤

def word_counts_featurizer(data):
    model = CountVectorizer()  # Convert a collection of text documents to a matrix of token counts
    X = model.fit_transform(data)
    print('# features: {}'.format(X.shape[1]))
    return X

def KNN(X_train, Y_train, X_test, Y_test, n_neighbors):
    knn = KNeighborsClassifier(n_neighbors = n_neighbors)
    knn_model_1 = knn.fit(X_train, Y_train)
    knn_prediction_train = knn_model_1.predict(X_train)
    knn_prediction_test = knn_model_1.predict(X_test)
    eval_train = evaluation_metric(small_Y_train, knn_prediction_train)
    eval_test = evaluation_metric(small_Y_test, knn_prediction_test)
    print('k-NN accuracy for test set: %f' % knn_model_1.score(X_test, Y_test))

    print (eval_train)
    print (eval_test)

small_X = word_counts_featurizer(small_df_selectedFeatures['review_body'])
small_Y = small_df_selectedFeatures['pos_neg']
small_X_train, small_X_test, small_Y_train, small_Y_test = train_test_split(small_X, small_Y, test_size=0.3, random_state=0)
KNN_model = KNN(small_X_train, small_Y_train, small_X_test, small_Y_test, n_neighbors=5)

# features: 26179
k-NN accuracy for test set: 0.794188
          accuracy  precision    recall   f_score  macro_f_score
observed                                                        
pos       0.834589   0.844704  0.329828  0.844704       0.839156
neg       0.834589   0.833609  0.982260  0.833609       0.839156
          accuracy  precision    recall   f_score  macro_f_score
observed                                                        
pos       0.794188   0.643204  0.215447  0.643204       0.724928
neg       0.794188   0.806652  0.964774  0.806652       0.724928


In [10]:
# 從這個cell開始, 我主要是想要create a new column. 
# 這個新col 內容和 review_body 一樣, 只是經過一些處理, 例如remove punctuation, lower the case and then split

def text_processing(data):
    data['review_body'] = data['review_body'].astype(str)

    #lower the case and then split
    data['review_body_1'] = START_SYMBOL + ' ' + data['review_body'] + ' ' +  END_SYMBOL
    data['review_body_2'] = data['review_body_1'].str.lower().str.split(' ', expand = False) 
    data.drop(['review_body_1'], axis=1)
    return data

# Note: if we want to remove punctuation, can use: df['col_name'].str.replace('[^\w\s]','') 
small_df_selectedFeatures = text_processing(small_df_selectedFeatures)
small_df_selectedFeatures.head(3)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,pos_neg,review_body_1,review_body_2
0,B000U92DLA,5,6,6,Y,A Must For On The Go,"Purchased the charger from Nextop2100, arrived...",1,"<s> Purchased the charger from Nextop2100, arr...","[<s>, purchased, the, charger, from, nextop210..."
1,B0008GCYNW,3,15,17,Y,The Good and the Not-so Good,I was attracted to this camera for three reaso...,0,<s> I was attracted to this camera for three r...,"[<s>, i, was, attracted, to, this, camera, for..."
2,B003CZ7L84,4,1,1,Y,Great for what we paid!,We took it with us to Cancun. My husband took ...,1,<s> We took it with us to Cancun. My husband t...,"[<s>, we, took, it, with, us, to, cancun., my,..."


In [14]:
small_df_selectedFeatures['review_body_2']

0        [<s>, purchased, the, charger, from, nextop210...
1        [<s>, i, was, attracted, to, this, camera, for...
2        [<s>, we, took, it, with, us, to, cancun., my,...
3        [<s>, this, bag, is, perfect, for, our, nikon,...
4        [<s>, crapy, cheap, plastic,, leg, broke, the,...
                               ...                        
18003    [<s>, great, picture, quaility, with, my, cano...
18004    [<s>, feels, very, weak,, but, does, the, tric...
18005    [<s>, i, bought, my, first, 339, dw, and, it, ...
18006    [<s>, the, picture, and, description, did, not...
18007    [<s>, this, is, my, second, time, purchasing, ...
Name: review_body_2, Length: 18008, dtype: object

In [11]:
# train, test = train_test_split(df, test_size=0.3)
# print('training dataset: ', train.shape)
# print('test dataset: ', test.shape)

small_train, small_test = train_test_split(small_df_selectedFeatures, test_size=0.3, random_state=0)
print('small training dataset: ', small_train.shape)
print('smalle test dataset: ', small_test.shape)


small training dataset:  (12605, 10)
smalle test dataset:  (5403, 10)


In [12]:
# 這邊是仿造 HW4 Q1 部分, 不過還沒研究怎麼套用到老師寫的model裡

small_train_text = small_train['review_body_2'].values.tolist()
small_train_vocab = sorted(set([item for sublist in small_train_text for item in sublist])) + [UNK_SYMBOL]

print(len(small_train_vocab))


52442


In [13]:
# Tokenizer (n-gram) 還沒想到怎麼用這個
import vsm
def tokenize_example(s):
    ngrams = vsm.get_character_ngrams(s, n=4)
    return [START_SYMBOL] + ngrams + [END_SYMBOL]

In [14]:
#loop through each review to get all the tokneizer for all the reviews
n_gram_rep = [tokenize_example(s) for s in small_train_text]

n_gram_rep_vocab = sorted({w for toks in n_gram_rep for w in toks}) + [UNK_SYMBOL]

In [15]:
#Glove embedding 按照老師的glove embedding不知如何放進去model還在研究中
import utils
from utils import START_SYMBOL, END_SYMBOL, UNK_SYMBOL
GLOVE_HOME = os.path.join('data', 'glove.6B')
glove_file = utils.glove2dict(os.path.join(GLOVE_HOME,'glove.6B.50d.txt'))

def create_glove_embedding(vocab, glove_base_filename='glove.6B.50d.txt'):
    
    glove_file = utils.glove2dict(os.path.join(GLOVE_HOME,'glove.6B.50d.txt'))
    glove_embedding = utils.create_pretrained_embedding(glove_file, vocab, required_tokens=('$UNK', "<s>", "</s>"))
    return glove_embedding

In [16]:
dev_glove_embedding, dev_glove_vocab = create_glove_embedding(n_gram_rep_vocab)

In [17]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import punkt
import os

In [18]:
#參照網路上例子的glove embedding
embeddings_index = {}
f = open('data/glove.840B.300d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    try:
       coefs = np.asarray(values[1:], dtype='float32')
       embeddings_index[word] = coefs
    except ValueError:
       pass
f.close()
print('Found %s word vectors.' % len(embeddings_index))

2196018it [08:07, 4505.09it/s]


Found 2195885 word vectors.


In [19]:
#word_tokenize and glove embedding and normalization
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [21]:
#Transform all the data to glove and sent2vec
xtrain_glove = [sent2vec(x) for x in tqdm(small_train_text)]

100%|███████████████████████████████████████████████████████████████████████████| 12605/12605 [00:35<00:00, 355.24it/s]


In [60]:
# logistic regression
clf = LogisticRegression()
clf.fit(xtrain_glove, small_train['pos_neg'])
predictions = clf.predict(xtrain_glove)
print (clf.score(xtrain_glove, small_train['pos_neg']))

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7755652518841729


In [87]:
def evaluation_metric(Truelabel, predictions, beta = 0):
    cm = metrics.confusion_matrix(Truelabel, predictions)
    classes = ['pos', 'neg']
    ex = pd.DataFrame(cm, columns=classes,index=classes)
    ex.index.name = "observed"
    
    
    eval = {}
    accuracy = ex.values.diagonal().sum() / ex.values.sum()
    precision = ex.values.diagonal() / ex.sum(axis=0)
    recall = ex.values.diagonal() / ex.sum(axis=1)
    f_score = (beta**2 + 1) * ((precision * recall) / ((beta**2 * precision) + recall))
    macro_f_score = f_score.mean(skipna=False)
    eval['accuracy'] = accuracy
    eval['precision'] = precision
    eval['recall'] = recall
    eval['f_score'] = f_score
    eval['macro_f_score'] = macro_f_score
    return pd.DataFrame.from_dict(eval)

In [88]:
evaluation_metric(small_train['pos_neg'], predictions)

,accuracy,precision,recall,f_score,macro_f_score
observed,,,,,
pos,0.775565,0.627660,0.020680,0.627660,0.702168
neg,0.775565,0.776677,0.996411,0.776677,0.702168


In [29]:
#想說要跟老師一樣把所以logistic parameter都跑一遍 部過pos recall很差還在想是哪裡可能用錯了
def fit_softmax_with_crossvalidation(X, y):
    """A MaxEnt model of dataset with hyperparameter 
    cross-validation. Some notes:
        
    * 'fit_intercept': whether to include the class bias feature.
    * 'C': weight for the regularization term (smaller is more regularized).
    * 'penalty': type of regularization -- roughly, 'l1' ecourages small 
      sparse models, and 'l2' encourages the weights to conform to a 
      gaussian prior distribution.
    
    Other arguments can be cross-validated; see 
    http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
    
    Parameters
    ----------
    X : 2d np.array
        The matrix of features, one example per row.
        
    y : list
        The list of labels for rows in `X`.   
    
    Returns
    -------
    sklearn.linear_model.LogisticRegression
        A trained model instance, the best model found.
    
    """    
    basemod = LogisticRegression(
        fit_intercept=True, 
        solver='liblinear', 
        multi_class='auto')
    cv = 5
    param_grid = {'fit_intercept': [True, False], 
                  'C': [0.4, 0.6, 0.8, 1.0, 2.0, 3.0],
                  'penalty': ['l1','l2']}    
    best_mod = utils.fit_classifier_with_crossvalidation(
        X, y, basemod, cv, param_grid)
    return best_mod